In [3]:
import urllib.request
f = urllib.request.urlretrieve("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz", "kddcup.data_10_percent.gz")

In [15]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
data_file = "./kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)

In [16]:
raw_data.count()

494021

In [17]:
raw_data.take(5)

['0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,19,19,1.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,29,29,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,219,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.00,0.00,1.00,0.00,0.00,39,39,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.00,0.00,1.00,0.00,0.00,49,49,1.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,normal.']

In [19]:
normal_raw_data = raw_data.filter(lambda x: 'normal.' in x)

In [23]:
from time import time
t0 = time()
normal_count = normal_raw_data.count()
tt = time() - t0
print('There are {} "normal" interactions'.format(normal_count))
print('Count completed in {} seconds'.format(tt))

There are 97278 "normal" interactions
Count completed in 1.1012539863586426 seconds


In [24]:
from pprint import pprint
csv_data = raw_data.map(lambda x: x.split(","))
t0 = time()
head_rows = csv_data.take(5)
tt = time() - t0
print("Parse completed in {} seconds".format(round(tt, 5)))

Parse completed in 0.06368 seconds


In [25]:
print(head_rows[0])

['0', 'tcp', 'http', 'SF', '181', '5450', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '8', '8', '0.00', '0.00', '0.00', '0.00', '1.00', '0.00', '0.00', '9', '9', '1.00', '0.00', '0.11', '0.00', '0.00', '0.00', '0.00', '0.00', 'normal.']


In [26]:
def parse_interation(line):
    elems = line.split(',')
    tag = elems[41]
    return tag, elems

key_csv_data = raw_data.map(parse_interation)
head_rows = key_csv_data.take(5)
print(head_rows[0])

('normal.', ['0', 'tcp', 'http', 'SF', '181', '5450', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '8', '8', '0.00', '0.00', '0.00', '0.00', '1.00', '0.00', '0.00', '9', '9', '1.00', '0.00', '0.11', '0.00', '0.00', '0.00', '0.00', '0.00', 'normal.'])


In [27]:
t0 = time()
all_data = raw_data.collect()
tt = time() - t0
print('Data collected in {} seconds'.format(tt))

Data collected in 5.687464952468872 seconds


In [31]:
# get data from file
data_file = "./kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)

# parse into key-value pairs
key_csv_data = raw_data.map(parse_interation)

# filter normal key interations
normal_key_interactions = key_csv_data.filter(lambda x: x[0] == 'normal.')

# collect all
t0 = time()
all_normal = normal_key_interactions.collect()
tt = time() - t0
normal_count = len(all_normal)
print('Normal data collected in {} seconds'.format(round(tt, 3)))
print('There are {} normal interactions in total'.format(normal_count))

Normal data collected in 3.951 seconds
There are 97278 normal interactions in total
